In [1]:
import os
import urllib
import pandas as pd
from skimage import io
from tqdm import tqdm
import numpy as np
from params import dresden_images_root, dresden_csv, images_db_path

In [2]:
path_list = []
brand_model_list = []

In [27]:
# Create output folder if needed
if not os.path.exists(dresden_images_root):
    os.makedirs(dresden_images_root)

data = pd.read_csv(dresden_csv)
data = data[(data['brand'] == 'Canon') & (data['instance'] == 0)]

In [ ]:
# if the size of all images are not zero, then append to the list
if all(img.shape[:2]):
    count += 1
    brand_model = '_'.join([brand, model])
    brand_model_list.append(brand_model) 
    path_list.append(filename)

In [ ]:
# create a images database as a dictionary
images_db = {
    'brand_model': np.asarray(brand_model_list).flatten(),
    'path': np.asarray(path_list).flatten(),
}
np.save(images_db_path, images_db)

print('Completed.')

In [4]:
csv_rows = []
for i in range((data.shape[0])): 
    csv_rows.append(list(data.iloc[i, :])) 

In [5]:
count = 0
for csv_row in tqdm(csv_rows):

    filename, brand, model = csv_row[0:3]
    url = csv_row[-1]

    file_path = os.path.join(dresden_images_root,filename)

    try:
        if not os.path.exists(file_path):
            print('Downloading {:}'.format(filename))
            urllib.request.urlretrieve(url, file_path)

        # Load the image and check its dimensions
        img = io.imread(file_path)

        if img is None or not isinstance(img, np.ndarray):
            print('Unable to read image: {:}'.format(filename))
            # removes (deletes) the file path
            os.unlink(file_path)

        # if the size of all images are not zero, then append to the list
        if all(img.shape[:2]):
            count += 1
            brand_model = '_'.join([brand, model])
            brand_model_list.append(brand_model) 
            path_list.append(filename)

        else:
            print('Zero-sized image: {:}'.format(filename))
            os.unlink(file_path)

    except IOError:
        print('Unable to decode: {:}'.format(filename))
        os.unlink(file_path)

    except Exception as e:
        print('Error while loading: {:}'.format(filename))
        if os.path.exists(file_path):
            os.unlink(file_path)

print('Number of images: {:}'.format(len(path_list)))

print('Saving db to: {:}'.format(images_db_path))

# create a images database as a dictionary
images_db = {
    'brand_model': np.asarray(brand_model_list).flatten(),
    'path': np.asarray(path_list).flatten(),
}
np.save(images_db_path, images_db)

print('Completed.')